In [ ]:
import os
import sys
os.environ["PYSPARK_PYTHON"] = "/opt/continuum/anaconda/bin/python"
os.environ["JAVA_HOME"] = "/usr/java/jdk1.7.0_67-cloudera/jre"
os.environ["SPARK_HOME"] = "/opt/cloudera/parcels/CDH/lib/spark"
os.environ["PYLIB"] = os.environ["SPARK_HOME"] + "/python/lib"
sys.path.insert(0, os.environ["PYLIB"] +"/py4j-0.9-src.zip")
sys.path.insert(0, os.environ["PYLIB"] +"/pyspark.zip")

In [ ]:
conda install -c conda-forge/label/cf201901 pyspark

In [ ]:
conda install -c cyclus java-jdk

In [ ]:
from pyspark import SparkConf, SparkContext
sc = SparkContext(conf=SparkConf())

In [ ]:
from collections import defaultdict,OrderedDict
from pyspark.mllib.linalg import Vector, Vectors
from pyspark.mllib.clustering import LDA, LDAModel
import re

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql import Row
spark = SparkSession \
    .builder \
    .appName("DataFrame Intro") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()
sc1 = spark.sparkContext

In [ ]:
#raw = sc.textFile('Documents/BigDataTech/trump.txt')
df = spark.read.format("csv").option("header", "false").load("Documents/BigDataTech/trump.csv")
df.take(1)

In [ ]:
tweets = df[['_c1']]
tweets.take(1)

In [ ]:
tmp = tweets.rdd.flatMap(list)
tmp.take(5)

In [ ]:
dtrump = tmp.map(lambda x: re.sub('[,/_\."“;!*&()?”]','',x))
dtrump.take(5)

In [ ]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords 

In [ ]:
something = dtrump.flatMap(lambda x: x.split(','))
words = something.flatMap(lambda x: x.split(' ')).filter(lambda x: len(x)>1).map(lambda x: x.lower())

In [ ]:
stop_words = set(stopwords.words('english'))
finals = words.filter(lambda x: x not in stop_words).map(lambda x: (x,1))

In [ ]:
dt = dtrump.flatMap(lambda x: x.split(' ')).map(lambda x: x.lower()).filter(lambda x: x not in stop_words).filter(lambda x: len(x)>1).distinct().zipWithIndex()

In [ ]:
mydictionary = dt.collectAsMap()

In [ ]:
new_dct = dict()
for k,v in mydictionary.items(): new_dct[v]=k

In [ ]:
sorted_keys, sorted_values = zip(*sorted(mydictionary.items()))

In [ ]:
def to_sparseVector(doc): 
    return [id, Vectors.sparse(len(new_dct), 
                               sorted_keys, 
                               sorted_values)]

In [ ]:
corpus = dt.map(to_sparseVector)

In [ ]:
num_topics = 5
num_words_in_topic = 10

lda_model = LDA.train(corpus, k=num_topics, maxIterations=50)
topics = lda_model.describeTopics(maxTermsPerTopic=num_words_in_topic)

for idx in range(num_topics):
    print("Topic #{0}".format(idx))
    for i in range(num_words_in_topic):
        print("  {0}\t{1}".format(new_dct[topics[idx][0][i]], topics[idx][1][i]))

    print("")

print("Vocabulary size = {0}".format(len(mydictionary)))